In [1]:
import numpy as np
import matplotlib.pyplot as plt
from ase.io import read

In [ ]:
atoms = read("../structures/unrelaxed/scatt.xyz")
z_threshold = 30

symbols = atoms.get_chemical_symbols()
z_coords = atoms.positions[:, 2]

num_atoms_leads = np.sum(z_coords < z_threshold)
num_atoms_per_lead = num_atoms_leads // 2
total_atoms = len(atoms)
num_atoms_bridge = total_atoms - num_atoms_leads


print("total_atoms", total_atoms)
print("num_atoms_bridge", num_atoms_bridge)
print("num_atoms_leads", num_atoms_leads)

num_spherical_harmonics = {"C": 9, "H": 4}

# Partition atoms based on x-sorted assumption
left_lead_indices = np.arange(0, num_atoms_per_lead)
bridge_indices = np.arange(num_atoms_per_lead, num_atoms_per_lead + num_atoms_bridge)
right_lead_indices = np.arange(num_atoms_per_lead + num_atoms_bridge, total_atoms)


print("Left lead indices range:", left_lead_indices.min(), left_lead_indices.max())
print("Bridge indices range:", bridge_indices.min(), bridge_indices.max())
print("Right lead indices range:", right_lead_indices.min(),right_lead_indices.max())

total_sph_left_lead = sum(num_spherical_harmonics[symbols[i]] for i in left_lead_indices)
total_sph_bridge = sum(num_spherical_harmonics[symbols[i]] for i in bridge_indices)
total_sph_right_lead = sum(num_spherical_harmonics[symbols[i]]
                           for i in right_lead_indices)

# Populate nodes
nodes = [
    0,
    total_sph_left_lead,
    total_sph_left_lead + total_sph_bridge,
    total_sph_left_lead + total_sph_bridge + total_sph_right_lead
]

print("Number of orbitals in left lead = ", nodes[1] - nodes[0])
print("Number of orbitals in bridge = ", nodes[2] - nodes[1])
print("Number of orbitals in right lead = ", nodes[3] - nodes[2])

np.save("../unrelaxed/output/no_lowdin/device/nodes.npy",nodes)


total_atoms 1216
num_atoms_bridge 352
num_atoms_leads 864
Left lead indices range: 0 431
Bridge indices range: 432 783
Right lead indices range: 784 1215
Number of orbitals in left lead =  3888
Number of orbitals in bridge =  2848
Number of orbitals in right lead =  3888


In [ ]:
atoms = read("../structures/unrelaxed/bridge_sorted.xyz")
z_threshold = 30

symbols = atoms.get_chemical_symbols()
z_coords = atoms.positions[:, 2]

num_atoms_bridge = len(atoms)


print("num_atoms_bridge", num_atoms_bridge)

num_spherical_harmonics = {"C": 9, "H": 4}

# Partition atoms based on x-sorted assumption
bridge_indices = np.arange(0, num_atoms_bridge)


print("Bridge indices range:", bridge_indices.min(), bridge_indices.max())

total_sph_bridge = sum(num_spherical_harmonics[symbols[i]] for i in bridge_indices)

# Populate nodes
nodes = [
    0,
     total_sph_bridge,
]
nodes

In [ ]:
np.save("../unrelaxed/output/no_lowdin/bridge/nodes.npy",nodes)